In [ ]:
!pip install tensorflow==2.4.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2, MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model

In [ ]:
batch_size = 32
img_height = 244
img_width = 244
input_shape = (img_width,img_height)
num_classes=2

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))



In [ ]:
def my_net(model,freeze_layers=10,full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(2,activation='sigmoid')(x)
    model_final = Model(model.input,out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [ ]:
#model = ResNet50V2(weights='imagenet',include_top=False)
#model = my_net(model)

In [ ]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(244,244,3), alpha=1.0, include_top=False,
    input_tensor=None, pooling=None, classes=2,
    classifier_activation='softmax'
)
model = my_net(model)

In [ ]:
#model = tf.keras.applications.VGG16(
#    include_top=True, weights=None, input_tensor=None,
#    input_shape=(244, 244, 3), pooling=None, classes=2,
#    classifier_activation='softmax'
#)
#model = my_net(model)

In [ ]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=10,
    validation_data=val_ds
)

In [ ]:
!rm -Rf model

In [ ]:
model.save('model')
